In [29]:
import spacy
import glob # to import the email dataset forlder
import re # to use regular expression
import nltk # to use natural langauge prossening features
from nltk.tokenize import word_tokenize # to split words and sentences
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
from collections import namedtuple
from textblob import TextBlob # to use textblob algorthem to preprosseing the data and to get sentement of the email
from nltk import ne_chunk, pos_tag # to use part of speech tagging
from nltk.tree import Tree
from pprint import pprint # to print the output in orgnazied way
import json
import rake
import summarizer
import pandas as pd

In [30]:
email_dic ='emails_datasetNEW_s/*'
fields =('Sentiment','keyWord1','KeyWord2','KeyWord3','summary','EmailSender','EmailReceiver','EmailCopy','URL','Date','PhoneNumber', 'entity')
emailDTO  = namedtuple('emailDTO',fields)
finalResult = {}
#nlp = spacy.load('en_core_web_sm')
f = open("stopword.txt", "r")
read_ = f.read()
stop_words = read_.split("\n")
lemmatizer = WordNetLemmatizer()

In [31]:
###################################################################################################
#this code processes unstructured email data based on 2 phases - cleansing and extraction.
emailRX = re.compile(("To: ([a-z0-9!#$%&'*+\/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+\/=?^_`"
                    "{|}~-]+)*(@|\sat\s)(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?(\.|"
                    "\sdot\s))+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?)"))

In [32]:
# Cleaning
all_data_lst = []
for f in glob.glob('emails_datasetNEW/*'):
    with open(f, 'r') as content_file:
        content = content_file.read()
        for word in stop_words:
            if word in content: 
                content = content.replace(" "+ word + " ", " ")
        #print("\n\n\nfile : ",f,"\n\n\n") 
        #print(content)
        #doc = nlp(content)
        #print(doc)
        all_data_lst.append(content)

In [33]:
#Word Tokenization
tokenization_lst = []
for cont in content:
    tokenization_lst.append(word_tokenize(cont))

In [34]:
# lemmatization 
lemmatization_lst = []
for wordTokenize in tokenization_lst:
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in wordTokenize])
    lemmatization_lst.append(lemmatized_output)

In [35]:
# EvaluateSentiment
for f in glob.glob(email_dic):
    with open(f, 'r') as content_file:
        content = content_file.read()
        content = content.replace('\n',' ').replace('\r',' ').replace(',', ' ')
        blob = TextBlob(content)
        polarity =0
        for sentence in blob.sentences:
            polarity +=sentence.sentiment.polarity
        if polarity<0:
            polarity = -1
        elif polarity>0:
            polarity = 1
        elif polarity == 0:
            polarity = 0
        finalResult[f]=emailDTO(Sentiment=polarity,keyWord1='NA',KeyWord2='NA',KeyWord3='NA',summary='NA',EmailSender='NA',URL='NA',EmailCopy='NA',Date='NA',EmailReceiver='NA', PhoneNumber='NA',entity=[])
        #print(content.index, polarity)

In [36]:
for f in glob.glob(email_dic):
    with open(f, 'r') as content_file:
        content_file.seek(540)
        content1 = content_file.readlines()
        contlst=[]
        # to read just the email body
        for i in content1:
            if('Message-ID:' in i):
                a=1                         
            elif('Date:' in i):
                a=1
            elif('From:' in i):
                a=1
            elif('To:' in i):
                a=1
            elif('Subject:' in i):
                a=1
            elif('CC:' in i):
                a=1
            elif('Mime-Version:' in i):
                a=1
            elif('Content-Type:' in i):
                a=1
            elif('Content-Transfer-Encoding:' in i):
                a=1
            elif('Bcc:' in i):
                a=1
            elif('X-' in i):
                a=1
            elif('@' in i):
                a=1
            else:
                contlst.append(i)

        content=" ".join(contlst)
        content = content.replace('\n',' ').replace('\r',' ').replace(',', ' ')
        rake_obj = rake.Rake("stopword.txt", 3, 2, 2)
        #Each word has at least 2 characters  #Each phrase has at most 3 words #Each keyword appears in the text at least 2 times
        keywords = rake_obj.run(content)
        obj=finalResult[f]
        finalResult[f] = emailDTO(Sentiment=obj.Sentiment,keyWord1= keywords[0][0] if len(keywords)>=1 else 'NA',KeyWord2=keywords[1][0] if len(keywords) >=2 else 'NA',KeyWord3=keywords[2][0] if len(keywords) >=3 else 'NA',summary='NA',EmailSender='NA',URL='NA',EmailCopy='NA',Date='NA',EmailReceiver='NA', PhoneNumber='NA',entity=[])

In [37]:
for f in glob.glob(email_dic):
    with open(f, 'r') as content_file:
        content1 = content_file.readlines()
        contlst=[]
        # to read just the email body
        for i in content1:
            if('Message-ID:' in i):
                a=1                         
            elif('Date:' in i):
                a=1
            elif('From:' in i):
                a=1
            elif('To:' in i):
                a=1
            elif('Subject:' in i):
                a=1
            elif('CC:' in i):
                a=1
            elif('Mime-Version:' in i):
                a=1
            elif('Content-Type:' in i):
                a=1
            elif('Content-Transfer-Encoding:' in i):
                a=1
            elif('Bcc:' in i):
                a=1
            elif('X-' in i):
                a=1
            elif('@' in i):
                a=1
            else:
                contlst.append(i)
                
        content=" ".join(contlst)
        summeryResponse  = summarizer.summarize_txt(content,1)
        resp = 'NA'
        obj=finalResult[f]
        if len(summeryResponse)>=1:
            resp = " ".join(summeryResponse)
        finalResult[f] = emailDTO(Sentiment=obj.Sentiment,keyWord1=obj.keyWord1,KeyWord2=obj.KeyWord2,KeyWord3=obj.KeyWord3,summary=resp,EmailSender='NA',URL='NA',EmailCopy='NA',Date='NA',EmailReceiver='NA', PhoneNumber='NA',entity=[])

In [38]:
# Extract regex, fields
FROMEMAILRX = re.compile("From: ([a-z0-9_\.-]+@[\da-z\.-]+\.[a-z\.]{2,6})")
TOEMAILRX = re.compile("To: ([a-z0-9_\.-]+@[\da-z\.-]+\.[a-z\.]{2,6})")
CCEMAILRX = re.compile("cc: ([a-z0-9_\.-]+@[\da-z\.-]+\.[a-z\.]{2,6})")
URIRX = re.compile(("https?://\S+"))
DATERX = re.compile("Date: (\w+), ([0-9]+) (\w+) ([0-9]+)")
PHONERX = re.compile("(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})")
for f in glob.glob(email_dic):
    with open(f, 'r') as content_file:
        content1 = content_file.readlines()
        contlst=[]

        for i in content1:
            if('Message-ID:' in i):
                a=1
            elif('Mime-Version:' in i):
                a=1
            elif('Content-Type:' in i):
                a=1
            elif('Content-Transfer-Encoding:' in i):
                a=1
            else:
                contlst.append(i)
        content=" ".join(contlst)
        obj=finalResult[f]
        EmailSender =re.findall(FROMEMAILRX,content)
        URL =re.findall(URIRX,content)
        EmailCopy = re.findall(CCEMAILRX,content)
        Date = re.findall(DATERX,content)
        EmailReceiver = re.findall(TOEMAILRX,content)
        PhoneNumber = re.findall(PHONERX, content)
        finalResult[f] = emailDTO(Sentiment=obj.Sentiment,keyWord1=obj.keyWord1,KeyWord2=obj.KeyWord2,KeyWord3=obj.KeyWord3,summary=obj.summary,EmailSender=EmailSender[-5:],URL=URL[-5:],EmailCopy=EmailCopy[-5:],Date=Date[-5:],EmailReceiver=EmailReceiver[-5:], PhoneNumber=PhoneNumber[-5:],entity=[])

In [39]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\amahlawi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\amahlawi\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\amahlawi\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [40]:
# extract entity field
for f in glob.glob(email_dic):
    with open(f, 'r') as content_file:
        content_file.seek(640)
        content = content_file.read().replace('\n','  ')
        obj=finalResult[f]
        chunk = ne_chunk(pos_tag(word_tokenize(content)))
        cont_chunk = []
        curr_chunk = []
        named_entities = []
        for i in chunk:
            if type(i) == Tree:
                curr_chunk.append(" ".join([token for token, pos in i.leaves()]))
            elif curr_chunk:
                named_entity = " ".join(curr_chunk)
                if named_entity not in cont_chunk:
                    cont_chunk.append(named_entity)
                    curr_chunk = []
                #doc = ''
                #for i in cont_chunk[:10]:
                    #doc += str(i)
                #doc1 = nlp(doc)
                #if doc1.ents:
                    #for ent in doc1.ents:
                        #named = {ent.text+' - '+ent.label_+' - '+str(spacy.explain(ent.label_))}
                        #named_entities.append(named)
                else:
                    continue

        finalResult[f] = emailDTO(Sentiment=obj.Sentiment,keyWord1=obj.keyWord1,KeyWord2=obj.KeyWord2,KeyWord3=obj.KeyWord3,summary=obj.summary,EmailSender=obj.EmailSender,EmailReceiver=obj.EmailReceiver,EmailCopy=obj.EmailCopy,URL=obj.URL,Date=obj.Date,PhoneNumber=obj.PhoneNumber,entity=cont_chunk[:10])
                                                                    

In [42]:
resultMap = {}
for key,val in finalResult.items():
    resultMap[key] = val._asdict()
#print (pprint(json.dumps(resultMap)))

In [25]:
df = pd.DataFrame.from_dict(resultMap, orient="index")
df['index'] = df.reset_index().index
df.set_index('index')

,Sentiment,keyWord1,KeyWord2,KeyWord3,summary,EmailSender,EmailReceiver,EmailCopy,URL,Date,PhoneNumber,entity
index,,,,,,,,,,,,
0,1,lot,work,netco,"(ii)\tBudget (due Jan 3, 2002)\t\t\t\t\t\tTo b...",[louise.kitchen@enron.com],[wes.colwell@enron.com],[john.lavorato@enron.com],[],"[(Mon, 31, Dec, 2001)]",[],"[mitch.robinson, NETCO, Kitchen, Louise, Colwe..."
1,1,NA,NA,NA,Please execute and send\n to: Grande Communic...,[hunter.williams@grandecom.com],[gthorse@keyad.com],[k..allen@enron.com],[],"[(Tue, 11, Sep, 2001)]","[512-757-2794, 512-878-5467]","[PALLEN, Attached, Grande Communications Servi..."
2,1,intended recipient,wink,bid,512.505.3709\n Fax 512.505.3711\n \n \n -----O...,[richard.morgan@austinenergy.com],[k..allen@enron.com],[],[],"[(Wed, 10, Oct, 2001)]","[704-1194, 512.505.3709, 512.505.3711, 713-853...","[Wink, Richard Morgan Manager, Green Building ..."
3,1,copy,NA,NA,I don't know how many pages it will be but le...,[wise.counsel@lpl.com],[k..allen@enron.com],[],[],"[(Thu, 25, Oct, 2001)]",[],[]
4,1,resolution center,transfer,continue,The following options are also available throu...,[no.address@enron.com],[],[],[],"[(Thu, 27, Dec, 2001)]","[888-877-7757, 800-973-6766, 713-345-4745, 888...","[Resolution Center, Toll Free, ENW, Enron Buil..."
